# Syncing

## Git remote
The git remote command lets you create, view, and delete connections to other repositories. Remote connections are more like **bookmarks** rather than direct links into other repositories. Instead of providing real-time access to another repository, they serve as convenient names that can be used to reference a not-so-convenient URL.

To view the current remote configurations:
```bash
> git remote
origin
> git remote -v
origin	https://github.com/yuzaiusa/git_study_notes.git (fetch)
origin	https://github.com/yuzaiusa/git_study_notes.git (push)
```

To manage a remote bookmark,
```bash
git remote add <name> <url>
git remote rm <name>
git remote rename <old_name> <new_name>
```

When you clone a repository with `git clone`, it automatically creates a remote connection called `origin` pointing back to the cloned repository. This is useful for developers creating a **local copy** of a central repository, since it provides an easy way to pull upstream changes or publish local commits. This behavior is also why most Git-based projects call their central repository origin.